# 05DK. Hex-grid correlations, Denmark-wide
## Project: Bicycle node network loop analysis

This notebook puts data into an H3 grid and performs correlations.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2025-08-01  
Last modified: 2025-08-01  

## To do

- [ ] Is node density a proxy for loop census size?

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity

In [ ]:
print("Running scenario in " + STUDY_AREA)
for k, v in SCENARIO[SCENARIOID].items():
    print(k + ": " + str(v))

## Functions

In [ ]:
%run -i functions.py

## Load data

This can take several minutes.

In [ ]:
if load_data:
    if LOOP_LENGTH_BOUND:
        llb_string = "_maxlength" + str(LOOP_LENGTH_BOUND)
    else:
        llb_string = ""

    with open(
        PATH["data_out"]
        + "loopcensus_"
        + str(LOOP_NUMNODE_BOUND)
        + llb_string
        + ".pkl",
        "rb",
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        Gnx = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

In [ ]:
# Create gdf and igraph versions
nodes, edges = momepy.nx_to_gdf(net=Gnx, points=True, lines=True)
nodes.set_crs(epsg=25832, inplace=True)
edges.set_crs(epsg=25832, inplace=True)
G = ig.Graph.from_networkx(Gnx)
G.summary()

In [ ]:
# Plot network
if debug:
    plot_dk_gdf(
        nodes,
        edges,
        scale=0.4,
        vertex_size=get_vertex_size_constant(G.vcount()),
        link_width=get_edgewidth_constant(G.ecount()),
    )

## Put into H3 grid

In [ ]:
nodes.plot.scatter(x="x", y="y", style=".", alpha=0.5, figsize=(5, 5));

In [ ]:
nodes_nodata = nodes.drop(
    columns=["name", "id_cykelknudepkt", "_igraph_index", "x", "y", "nodeID"]
)  # drop all data
nodes_nodata.to_crs(epsg=4326, inplace=True)  # reproject for H3
nodes_nodata.head()

In [ ]:
nodesh3 = nodes_nodata.assign(count=1).h3.geo_to_h3_aggregate(6)
nodesh3.head()

In [ ]:
nodesh3.plot(column="count", figsize=(5, 5), legend=True)
plt.title("Node density");